In [1]:
from local_edc import print_info
print_info("usecase-ndvi")


***Notebook Title***  
EDC NDVI Use-case

***Notebook Description***  
Euro Data Cube Use-case 01: NDVI


***Notebook Dependencies***  
This notebook requires an active subscription to:
* EDC Sentinel Hub


In [2]:
from local_edc import setup_environment_variables
setup_environment_variables()

API credentials have automatically been injected for your active subscriptions.  
The following environment variables are now available:
* `SH_INSTANCE_ID`
* `SH_CLIENT_ID`
* `SH_CLIENT_SECRET`
* `SH_CLIENT_NAME`
* `GEODB_API_SERVER_URL`
* `GEODB_AUTH_CLIENT_ID`
* `GEODB_AUTH_AUD`
* `GEODB_AUTH_CLIENT_SECRET`
* `GEODB_API_SERVER_PORT`
* `GEODB_AUTH_DOMAIN`



In [3]:
from local_edc import check_compatibility
check_compatibility("v0.18.4")

Notebook is compatible


# NDVI timeline

This notebook presents an example on how a user can exploit the xcube client library to access and analyse data served via Euro Data Cube service API.

Thematically, this notebook illustrates how to generate a timeline of NDVI values calculated from Sentinel-2 bands. The normalized difference vegetation index (NDVI) is a simple graphical indicator that can be used to analyze remote sensing measurements acquired by a space platform, assessing whether or not the target being observed contains live green vegetation.

It demonstrates how xcube tools and Euro Data Cube complement each other. While Euro Data Cube services are used to access the data, xcube tools are utilized for analysis and visualization.

Specifically, following actions are demonstrated by this notebook:
1. Defining a new data cube for a given time range, region, and spatial resolution. 
2. Adding a new data layer to cube - calculated value of NDVI
3. Filling the cube with the data: requesting satellite imagery and calcualting NDVI layer
4. Displaying imageries, time series and linear plots for selected bands, area-of-interest and time

#### Importing dependancies
The user needs to first import the dependancies.

In [4]:
# xcube_sh imports
from xcube_sh.cube import open_cube
from xcube_sh.config import CubeConfig
from xcube_sh.observers import Observers
from xcube_sh.viewer import ViewerServer

# xcube imports
from xcube.core.maskset import MaskSet
from xcube.core.geom import mask_dataset_by_geometry
from xcube.core.geom import clip_dataset_by_geometry

# Various utilities
import json
import xarray as xr
import shapely.geometry
import IPython.display
import zarr
import numpy as np

To present the timeline of NDVI values, you will need to create a linear chart. 
 
To create the chart, it´s neccessary to import the Matplot library (matplotlib) first. The matplotlib is a plotting library which renders the plots.

In [5]:
import matplotlib.pyplot as plt

#### Specification of a geographical area of interest
The next step is to specify the area of interest - i.e. a geographical area, which the cube should cover and for which the NDVI will be calculated. 

The area of interest is specified via bounding box. You have to specify geographical coordinates of each corner of this bounding box.

For this demo, we are focussing on small coastal area near Kiel in Northern Germany (Baltic Sea).

In [6]:
x1 = 10.00 # degree
y1 = 54.27  # degree
x2 = 10.40  # degree
y2 = 54.50  # degree

bbox = x1, y1, x2, y2

 #### Visualisation of area of interest
 To check whether the area of interest has been specified correctly, you can integrate a simple map allowing visualisation of this area.

Visualize the bounding box.

In [7]:
IPython.display.GeoJSON(shapely.geometry.box(*bbox).__geo_interface__)

<IPython.display.GeoJSON object>

 #### Setting the spatial resolution of the cube
 The next step is to specify the spatial resolution (pixel size) on which the raster data (in this case Sentinel-2 imageries) should be resampled and stored in.
 
In this case, the desired resolution is roughly 10 meters per pixel:


In [8]:
plt.plot(timeseriesB08)
plt.xticks(rotation='vertical')
plt.figsize=(15, 6)
plt.show()

NameError: name 'timeseriesB08' is not defined

#### Displaing cube data for selected band and time step as imageries

We can also display the data for any combination of selected spectral band and time step of the times serie contained in the cube as an imagery. We may need this to check the data visually, over entire area of interest (geographical extent of the cube).
For example we'll select the time step 21.5.2018 and display the data as image for both spectral bands contianed in the cube (B04 RED and B08 NIR).

In [ ]:
cube.B04.sel(time='2018-05-21 10:00:00', method='nearest').plot.imshow(vmin=0.0, vmax=0.2, cmap='gray', figsize=(16, 10))

In [ ]:
cube.B08.sel(time='2018-05-21 10:00:00', method='nearest').plot.imshow(vmin=0.2, vmax=0.6, cmap='gray', figsize=(16, 10))

#### Displaing statistics for SentinelHub request
Now SentinelHub data requests have been made. We can easilly display statistics on this request. 


In [ ]:
request_collector.stats

#### Preparing values for new NDVI layer
Now we will prepare values for the new NDVI layer, based on combination of the bands already contained in our cube.

We are going to compute NDVI indexes from bands B04 and B08 of Sentinel-2 (S2L2A), using following well-know prescription:

Normalized Difference Vegetation Index (NDVI) = (NIR-RED)/(NIR+RED). In our case, Sentinel-2 bandwise speaking, NDVI = (B08-B04)/(B08+B04).


In [ ]:
ndvi=(cube.B08-cube.B04)/(cube.B04+cube.B08)

#### Specifying properties of new NDVI layer
We can also specify some properties of the new NDVI layer, which should be added into our cube - in this case, we set the "long name" and units of the layer.


In [ ]:
ndvi.attrs['long_name']='Normalized Difference Vegetation Index'
ndvi.attrs['units']='unitless'

#### Adding a new NDVI layer to cube
Now the values are calculated and the properties specified, we can easilly add the new NDVI layer into our cube.

In [ ]:
cube['NDVI']=ndvi

#### Displaying information for NDVI layer
Again, as in case of original bands 04 and 08, we can ceck the properties of the NDVI layer.

In [ ]:
cube.NDVI

#### Displayng time serie of NDVI
Also, we can display time serie of values of NDVI band at selected location for all time steps.

For example we'll select the point at latitude=54.40, longitude=10.1
Nearest-neighbor interpolation is used by xcube to find closest satellite pixel at each time step.


In [ ]:
timeseriesNDVI = cube.NDVI.sel(lat=54.40, lon=10.1, method='nearest').to_series()
timeseriesNDVI

#### Displayng time serie of NDVI in linear plot
Now, when we computed the NDVI values for each step in the time serie, we can easily display them in linear plot, as in case of original Sentinel-2 bands.


In [ ]:
plt.plot(timeseriesNDVI)
plt.xticks(rotation='vertical')
plt.figsize=(15, 6)
plt.show()

Plotting time-series NDVI data produces a temporal curve that summarizes the various stages that green vegetation undergoes during a complete growing season. 
Such curves can be analyzed to extract key phenological variables, or metrics, about a particular season, such as the start, peak and end of the growing season. These characteristics may correspond directly to conventional, ground-based phenological events like growing (the NDVI values grows), mowing, harvesting or ploughing (NDVI values drops rapidly) and provide indications of ecosystem dynamics.

#### Displaying NDVI at selected time step as image

To check the NDVI values over whole area of interest, we can display the NDVI values for any selected time step of the time serie as image in selected color scale. 
For example we'll select the time step 21.5.2018.

In [ ]:
cube.NDVI.sel(time='2018-05-21 10:00:00', method='nearest').plot.imshow(vmin=-1, vmax=1, cmap='RdYlGn', figsize=(16, 10))
#cube.NDVI.sel(time='2018-06-25 10:00:00', method='nearest').plot.imshow(vmin=-1, vmax=1, cmap='RdYlGn', figsize=(16, 10))
#cube.NDVI.sel(time='2018-07-25 10:00:00', method='nearest').plot.imshow(vmin=-1, vmax=1, cmap='RdYlGn', figsize=(16, 10))

#### Displaying NDVI for all time steps in the serie as image

To check the NDVI values over whole area of interest for all time step in the time serie, 
we can display the NDVI values for all selected time steps of the serie as image in the same color scale beside each other. 
This allows us to visually compare the development of NDVI values over whole time serie and to compare various time steps with each other.

In [ ]:
img=cube.NDVI.plot.imshow(col='time', col_wrap=4, vmin=-1, vmax=1,cmap='RdYlGn')